# SetFitを用いたFew-Shot文書分類

![SetFit_architecture](https://github.com/huggingface/setfit/raw/main/assets/setfit.png)

- SetFit: 
    - 論文: [Efficient Few-Shot Learning Without Prompts](https://arxiv.org/abs/2209.11055)
    - リポジトリ: https://github.com/huggingface/setfit
- 分類ラベル: 2種類
- 訓練データ数: 16個（各クラス8個）


##ライブラリのインストール

In [1]:
!pip -q install setfit fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 KB 854.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## データセットのダウンロード

[多言語感情分類データセットの日本語スプリット（Amazonレビュー）](https://huggingface.co/datasets/tyqiangz/multilingual-sentiments/viewer/japanese/train)を利用する。

In [2]:
from datasets import load_dataset

dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  \
0  普段使いとバイクに乗るときのブーツ兼用として購入しました。見た目や履き心地は良いです。 しか...  negative   
1  十分な在庫を用意できない販売元も悪いですが、Amazonやら楽⚪︎が転売を認めちゃってるのが...  negative   
2  見た目はかなりおしゃれで気に入りました。2、3回持ち歩いた後いつも通りゼンマイを巻いていたら...  negative   
3  よくある部分での断線はしませんでした ただiphoneとの接続部で接触不良、折れました ip...  negative   
4  プラモデルの塗装剥離に使う為に購入 届いて早速使ってみた 結果 １ヶ月経っても未だに剥離出来...  negative   

                 source  
0  amazon_reviews_multi  
1  amazon_reviews_multi  
2  amazon_reviews_multi  
3  amazon_reviews_multi  
4  amazon_reviews_multi  
----------------------------------------------------------------------------------------------------


Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  \
0  味自体及び吸い心地は良いのだが、不良品が多過ぎる。私の場合５本のうち２本が蒸気も出ず、吸い込...  negative   
1             ホームボタン周りの気泡が全く抜けません。 返金をお願いしましたが、断られた。  negative   
2  新旧含めて4つのカーテンレールがあるのですが、使用出来るカーテンレールはありませんでした。 ...  negative   
3            予約注文でしたが、どこから特典であるpdfダウンロードすればよいのでしょうか…  negative   
4  前のレビューにもありましたが、片方が全く動きません。 返品しようにも、なんだかめんどくさいし...  negative   

                 source  
0  amazon_reviews_multi  
1  amazon_reviews_multi  
2  amazon_reviews_multi  
3  amazon_reviews_multi  
4  amazon_reviews_multi  
----------------------------------------------------------------------------------------------------


Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  \
0  購入、貼付け後2週間もたたないうちに、上側から剥がれてきて画面から浮いた状態になってしまった...  negative   
1  以下の3点の理由により、期待はずれの粗悪品。 ①他の方のレビューにもある通り、天板の外観が掲...  negative   
2  この商品の内容等確認した購入したのですが、そのとおりなかなか設定ができなく、知人にもお願いし...  negative   
3  テストした結果、4000mahのスマホ一回と30％分、スマホのバッテリー残量の表示が正しけれ...  negative   
4  前回の黒いドレッサーバッグの評判がよかったため、予約して購入しました。 ガッカリです。 ヨレ...  negative   

                 source  
0  amazon_reviews_multi  
1  amazon_reviews_multi  
2  amazon_reviews_multi  
3  amazon_reviews_multi  
4  amazon_reviews_multi  
----------------------------------------------------------------------------------------------------
Dataset multilingual-sentiments downloaded and prepared to /root/.cache/huggingface/datasets/tyqiangz___multilingual-sentiments/japanese/1.0.0/b7cdd8874d82679e59432edf79e074f595c4ad26d2e562eba4fb55f361691b07. Subsequent calls will reuse this dat

  0%|          | 0/3 [00:00<?, ?it/s]

## 訓練データ作成

各クラスのデータ数がデータ数が8個になるようにする（層化抽出）。

In [3]:
import json

class_labels = [0, 2]  # 1(neutral)は用いない
samples_per_class = 8

train_dataset = dataset["train"].shuffle(seed=42)

with open("/content/train.json", "w", encoding="utf-8") as f_out:
    for class_label in class_labels:
        class_data = train_dataset.filter(lambda x: x["label"] == class_label).select(range(samples_per_class))
        for data in class_data:
            f_out.write(json.dumps(data, ensure_ascii=False))
            f_out.write("\n")

train_dataset = load_dataset("json", data_files="/content/train.json")["train"].shuffle(seed=42)
eval_dataset = dataset["test"].filter(lambda x: x["label"] in class_labels)

print(f"train_dataset: {len(train_dataset)}")
for sample in train_dataset:
    print(sample)

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3a48dd19d7b34325/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

train_dataset: 16
{'text': '簡易ワックスは昔からこれを使っていて手軽でよく滑る！', 'source': 'amazon_reviews_multi', 'label': 0}
{'text': '次の商品は届かなかった。キャンセル処理もなかなかしてくれず絶対ここから買いません。', 'source': 'amazon_reviews_multi', 'label': 2}
{'text': '蓋が欠陥。飲み口ふさぐ用の部品のはめ込み穴が空いてなくすぐとれる', 'source': 'amazon_reviews_multi', 'label': 2}
{'text': '安かろう悪かろうです。 TVはが悪く暗く自動サーチなしでいちいちサーチしなおさなければダメ。 ナビとの切り替えはいちいちどちらかをOFFとしなければならない。 ナビはします。 買い直しです。安すぎるのはダメですね。', 'source': 'amazon_reviews_multi', 'label': 2}
{'text': 'こちらのiTunesカード200＄分を購入しました。 50＄４枚購入したはずが、届いたのは50＄1枚と25＄6枚です。 iTunesに読み込み無事200＄分入金できましだか、iTunes、ゲームでの購入はできませんでした。 それだけでなく、Apple IDも凍結され無効化。Apple IDが使用できなくなりました。 Appleサポートに問い合わせAppleセキュリティーチームが捜査を行い原因を調査したところ、使用したiTunesカードに問題がありその影響でIDが強制凍結及び無効化されている状態とのことでした。 AmazonでiTunesカードを購入した事を伝えたところiTunesカードの不正品の可能性を調査するとのことです。 現段階ではiTunesカードの不正品の可能性が高いと言うお話で、最終報告ではないので私の口からは不正品と断言することはできません。 ですが、購入したiTunesカードが原因でApple IDが凍結され無効化状態になってしまった事実は確定的です。 もしかしたら販売していた商品の一部に不正品が紛れ込んでいた可能性もあります。 現在はAppleセキュリティーチームからの最終報告を待っている状態です。 もし、購入したiTunesカ

## 訓練と評価

In [4]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer


model = SetFitModel.from_pretrained("sonoisa/sentence-bert-base-ja-mean-tokens-v2")

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20,
    num_epochs=2,
    # column_mapping={"sentence": "text", "label": "label"},
)
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num examples = 640
  Num epochs = 2
  Total optimization steps = 80
  Total train batch size = 16


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [5]:
metrics = trainer.evaluate()
print(metrics)

***** Running evaluation *****


{'accuracy': 0.8825}


## 学習済みモデルを保存

In [6]:
MODEL_DIR = "/content/trained_model"

!mkdir -p {MODEL_DIR}
!rm -rf {MODEL_DIR}/*

In [7]:
# save_pretrainedメソッドの実行でsentence transformersがエラーを発生するため、一時的に回避する。
# エラー：TypeError: unsupported operand type(s) for +: 'PosixPath' and 'str'

if False:
    trainer.model.save_pretrained(MODEL_DIR)
else:
    trainer.model._save_pretrained(MODEL_DIR)

## 学習済みモデルの利用

In [8]:
model = SetFitModel.from_pretrained(MODEL_DIR)

In [9]:
model(["スパイダーマンの映画が大好き！", "パイナップルがピザに乗っているなんて最悪🤮"])

tensor([0, 2])